## POS tagging using modified Viterbi

### Data Preparation

In [61]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [62]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print(nltk_data[:2])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [63]:
# Splitting into train and test
train_set, test_set = train_test_split(nltk_data,test_size=0.05, random_state =100)
print(len(train_set))
print(len(test_set))

3718
196


In [64]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95949

In [65]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
print(tokens[:10])
# vocabulary
V = list(set(tokens))
print(len(V))

['One', 'bright', 'sign', 'is', 'that', 'a', 'growing', 'number', 'of', 'women']
12106


In [66]:
# number of tags
T = list(set([pair[1] for pair in train_tagged_words]))
print(len(T))
print(T)

12
['ADP', 'X', 'NUM', 'ADV', 'PRON', 'ADJ', 'PRT', 'DET', '.', 'CONJ', 'NOUN', 'VERB']


### Build the vanilla Viterbi based POS tagger

In [67]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag):
    tag_list = [pair for pair in train_tagged_words if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag/count_tag)

In [68]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1/count_t1)

In [69]:
# creating t x t transition matrix of tags. Each column is t2, each row is t1. Thus M(i, j) represents P(tj given ti)
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)
        
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,ADP,X,NUM,ADV,PRON,ADJ,PRT,DET,.,CONJ,NOUN,VERB
ADP,0.017492,0.034984,0.061910,0.013357,0.069119,0.107389,0.001484,0.323969,0.039754,0.000848,0.321213,0.008481
X,0.144898,0.074433,0.002857,0.025393,0.055705,0.016505,0.184891,0.055229,0.162831,0.010316,0.062371,0.204571
NUM,0.035056,0.211824,0.184195,0.002674,0.001485,0.033571,0.026144,0.003862,0.118835,0.013072,0.352347,0.016934
ADV,0.119507,0.023302,0.031624,0.077230,0.015646,0.130160,0.014314,0.069907,0.135153,0.006991,0.031624,0.344541
PRON,0.023372,0.092720,0.007280,0.034100,0.007663,0.072031,0.012261,0.009195,0.040613,0.004981,0.211494,0.484291
ADJ,0.078624,0.020311,0.020803,0.004914,0.000491,0.067158,0.010156,0.004914,0.063882,0.016052,0.700901,0.011794
PRT,0.019357,0.013123,0.056102,0.010171,0.017717,0.083661,0.001969,0.101050,0.043635,0.002297,0.245735,0.405184
DET,0.009618,0.045323,0.021640,0.012623,0.003727,0.204977,0.000240,0.005771,0.017913,0.000481,0.637293,0.040394
.,0.092206,0.026908,0.081353,0.052292,0.065208,0.043681,0.002511,0.173558,0.092923,0.058032,0.222531,0.088708
CONJ,0.053778,0.008809,0.042188,0.053778,0.058414,0.118683,0.003709,0.118683,0.035698,0.000464,0.350487,0.155308


In [70]:
# Viterbi Heuristic
def Vanilla_viterbi(words):
    state = []
    T = list(set([pair[1] for pair in train_tagged_words]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            #emission_p = word_given_tag(words[key], tag)
            #word_tag_df
            emission_p = word_given_tag(words[key], tag)
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [71]:
# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

In [72]:
%%time
tagged_seq = Vanilla_viterbi(test_tagged_words)
print(tagged_seq[:10])

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

[('Investors', 'NOUN'), ('took', 'VERB'), ('advantage', 'NOUN'), ('of', 'ADP'), ('Tuesday', 'NOUN'), ("'s", 'PRT'), ('stock', 'NOUN'), ('rally', 'NOUN'), ('*-1', 'X'), ('to', 'PRT')]
Wall time: 9min 26s


### Solve the problem of unknown words

In [73]:
# specify patterns for tagging. example from the NLTK book


patterns = [
    (r'.*ing$', 'VERB'),              
    (r'.*ed$', 'VERB'),               
    (r'.*es$', 'VERB'),               
    (r'.*ly$', 'ADV'),              
    (r'.*ble$', 'ADJ'),                
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), 
    (r'(?<=[0-9])(?:st|nd|rd|th)$', 'NUM'), #ordinal Numbers
    (r'.*', 'NOUN')                    
]

regexp_tagger = nltk.RegexpTagger(patterns)

def rulebased_tagging(test_words):
    return regexp_tagger.tag([test_words])[0][1]

#### Evaluating tagging accuracy

In [74]:
# Updated Viterbi Heuristic

def Viterbi_probabilistic(words):
    state = []
    T = list(set([pair[1] for pair in train_tagged_words]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            #emission_p = word_given_tag(words[key], tag)
            #word_tag_df
            if(words[key] in V):
                emission_p = word_given_tag(words[key], tag)
                state_probability = emission_p * transition_p
            else:
                state_probability = transition_p
                                   
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [75]:
def Viterbi_rulebased(words):
    state = []
    T = list(set([pair[1] for pair in train_tagged_words]))
    for key, word in enumerate(words):
    #initialise list of probability column for a given observation
        p = []
        state_probability = 0
        rule_tag = ""
        if words[key] in V:
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]
                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)
                state_probability = emission_p * transition_p
                p.append(state_probability)
            pmax = max(p)
            state_max = T[p.index(pmax)]
            state.append(state_max)
            
        else:
            rule_tag=rulebased_tagging(words[key])
            state.append(rule_tag)
    return list(zip(words, state))

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [76]:
%%time
tagged_seq1 = Viterbi_probabilistic(test_tagged_words)
# accuracy
check = [i for i, j in zip(tagged_seq1, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print(accuracy)

0.9331499894224666
Wall time: 8min 44s


In [77]:
%%time
tagged_seq2 = Viterbi_rulebased(test_tagged_words)
# accuracy
check = [i for i, j in zip(tagged_seq2, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print(accuracy)

0.9481700867357732
Wall time: 8min 59s


### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [78]:
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import brown
# nltk.download('brown')
brown_news_tagged = brown.tagged_words(categories='news', tagset='universal')

import requests
response = requests.get('https://cdn.upgrad.com/UpGrad/temp/9dca5f3b-53c3-47e1-86d5-5ec5dafad6f0/Test_sentences.txt')
data = response.text

from nltk.tokenize import sent_tokenize, word_tokenize
sentense = sent_tokenize(data)
word_sample= word_tokenize(data)


# calculating tags using nltk.pos_tags, tagset universal

tagged_data=nltk.pos_tag(word_sample,tagset='universal')
# tagged_data
sample_sent =list([pair[0] for pair in tagged_data])
sample_sent[:5]
tagged_data[:5]

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Viswas\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Viswas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Viswas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN')]

In [79]:
# Original Vanilla Viterbi algorithm
tagged_seq3 = Vanilla_viterbi(sample_sent)
print(tagged_seq3[:5])

# accuracy
check = [i for i, j in zip(tagged_seq3, tagged_data) if i == j] 
accuracy = len(check)/len(tagged_seq3)
print(accuracy)

[('Android', 'ADP'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN')]
0.7624309392265194


In [80]:
# Modified viterbi using probablilistic algorithm
tagged_seq4 = Viterbi_probabilistic(sample_sent)
print(tagged_seq4[:5])

# accuracy
check = [i for i, j in zip(tagged_seq4, tagged_data) if i == j] 
accuracy = len(check)/len(tagged_seq4)
print(accuracy)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN')]
0.8674033149171271


In [82]:
# Modified viterbi using rule-based algorithm
tagged_seq5 = Viterbi_rulebased(sample_sent)
print(tagged_seq5[:5])

# accuracy
check = [i for i, j in zip(tagged_seq5, tagged_data) if i == j] 
accuracy = len(check)/len(tagged_seq5)
print(accuracy)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN')]
0.9281767955801105
